In [196]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import copy

In [176]:
page = requests.get("https://www.baseball-reference.com/leagues/MLB/2019-standard-batting.shtml")
soup = BeautifulSoup(page.content, 'html.parser')
my_table = soup.find('table')
my_head = my_table.find('thead')
my_head_cells = [cell.text for cell in my_head.find_all('th')]
my_rows = [row for row in my_table.find_all('tr')]
my_cells = [[float(cell.text) for cell in row.find_all('td')] for row in my_rows]
my_cells = [cell for cell in my_cells if cell]
stats = pd.DataFrame(data=my_cells, columns=my_head_cells[1:])
stats = stats.iloc[:30, :]

df = stats[['PA', 'H', '2B', '3B', 'HR', 'BB', 'GDP', 'HBP', 'SH', 'SF']]
df['1B'] = df['H'] - df[['2B', '3B', 'HR']].sum(1)
df['WALK'] = df[['BB', 'HBP']].sum(1)
df['OUT'] = df['PA'] - df[['H', 'WALK']].sum(1)

df_probs = df[['1B', '2B', '3B', 'HR', 'WALK', 'OUT']]
df_probs = df_probs.div(df_probs.sum(1), axis=0)

/home/jequezinho/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/jequezinho/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jequezinho/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

In [177]:
class Player:
    def __init__(self, probs):
        self.probs = pd.Series(probs)
        self.stats = []
        
    def at_bat(self):
        outcome = np.random.choice(self.probs.index, p=self.probs.values)
        self.stats.push(outcome)
        return outcome

In [ ]:
class Team:
    def __init__(self, players):
        self.players=players
        self.record = [0, 0]

    def update_record(self, boo):
        if boo:
            self.record[0] += 1
        else:
            self.record[1] += 1 

In [ ]:
class Game:
    def __init__(self,
                 teams,
                 inning=1,
                 outs=0,
                 away_or_home=0,
                 bases=[0,0,0],
                 score=[0,0],
                 game_on=True):
        self.teams=teams
        self.inning=inning
        self.outs=outs
        self.away_or_home=away_or_home
        self.bases=bases
        self.score=score
        self.game_on=game_on
        self.current_player=[0,0]

    def walker:
        self.bases.append(0)
        self.bases[0] += 1
        for i in range(3):
            if self.bases[i]==2:
                self.bases[i] -= 1
                self.bases[i+1] += 1
        runs = self.bases[-1]
        self.bases = self.bases[:3]
        self.score[self.away_or_home] += runs

    def hitter(self, hit_type):
        if hit_type == '1B':
            self.bases = [1,0].extend(self.bases)
        elif hit_type == '2B':
            self.bases = [0,1].extend(self.bases)
        elif hit_type == '3B':
            self.bases = [0,0,1].extend(self.bases)
        elif hit_type == 'HR':
            self.bases = [0,0,0,1].extend(self.bases)
        runs = sum(self.bases[3:])
        self.bases = self.bases[:3]
        self.score[self.away_or_home] += runs

    def at_bat(self):
        player=self.teams[self.away_or_home][self.current_player]
        result = player.at_bat()
        if result == 'OUT':
            self.outs += 1
        elif result == 'BB':
            self.walker()
        else:
            self.hitter(result)
        if (self.inning >= 9 and ((self.outs >= 3 and self.away_or_home == 0) or self.away_or_home == 1) and self.score[0] < self.score[1]) or
        (self.inning >= 9 and self.outs >= 3 and self.score[0] > self.score[1]):
            self.game_on = False
        if self.outs >= 3:
            if self.away_or_home == 1:
                self.inning += 1
            self.outs = 0
            self.current_player[self.away_or_home] = (self.current_player[self.away_or_home] + 1) % 9
            self.away_or_home = (self.away_or_home + 1) % 2
            self.bases = [0, 0, 0]

    def play_game(self):
        while self.game_on:
            self.at_bat()
        final_score = copy.copy(self.score)
        winner = 1 if (self.score[0] < self.score[1]) else 0
        self.inning = 1
        self.outs = 0
        self.away_or_home = 0
        self.bases = [0,0,0]
        self.score = [0,0]
        self.game_on = True
        return {
            "final_score": final_score,
            "winner": winner
        }

In [ ]:
class Simulator:
    def __init__(self, teams, inning=1, bases=[0,0,0], outs=0, score=[0,0]):
        self.teams=teams
        self.inning=1
        self.outs=0
        self.bases=[0,0,0]
        self.score=[0,0]
    
    def simulate(self, its=100):
        game_log = []
        wins = 0
        for i in range(its):
            game = Game([getattr(self, attr) if "__" not in attr])
            result = game.play_game()
            wins += result.winner
            game_log.append(result)
        print(f"The home team won ${wins} out of ${its}, for a winning percentage of {wins / its * 100}%!")
        return game_log

In [178]:
team1 = df_probs.sample(9)
avails = [ix for ix in df_probs.index if ix not in team1.index]
team2 = df_probs.iloc[avails, :].sample(9)

team1 = Team([Player(team1.iloc[i, :]) for i in range(team1.shape[0])])
team2 = Team([Player(team2.iloc[i, :]) for i in range(team2.shape[0])])

g = Game(teams=[team1,team2])

In [192]:
[(ho, getattr(g, ho)) for ho in dir(g) if "__" not in ho]

[('awayOrHome', 0),
 ('bases', [0, 0, 0]),
 ('gameOn', False),
 ('inning', 1),
 ('outs', 0),
 ('score', [0, 0]),
 ('teams',
  [<__main__.Team at 0x7f092e10a1d0>, <__main__.Team at 0x7f092e107668>])]